In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Read Data
df_rbkp = pd.read_excel('../Data1/RBKP.XLSX')

In [ ]:
df_rbkp.head(4)

In [ ]:
df_rseg = pd.read_excel('../Data1/RSEG.XLSX')

In [ ]:
df_rseg.head(4)

In [ ]:
print(df_rseg.shape)
print(df_rbkp.shape)

In [ ]:
df_re = pd.merge(df_rseg, df_rbkp, how='left', on='BELNR')

In [ ]:
#sanity check (first number should be the higher one from before, second should be addition of the right numbers)
df_re.shape

In [ ]:
df_re.head(5)

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_re)

# Data Cleansing

## initial cleaning

In [ ]:
df = df_re.copy()

In [ ]:
print(df.shape)
# Spalten die nur NAN enthalten entfernen
df = df.dropna(axis=1, how='all')

# Spalten die ausschließlich XY Werte enthalten (0 / X / 1 / etc.) dropen
df = df.loc[:, (df.astype(str) != 0).any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,0').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,00').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,000').any(axis=0)]
df = df.loc[:, (df.astype(str) != 'X').any(axis=0)]
df = df.loc[:, (df.astype(str) != '1').any(axis=0)]

#spaletn entfernen die immer die selben Werte enthalten
keep_columns = [col for col in df.columns if len(df[col].unique()) > 1]
df = df[keep_columns].copy()


#print(len(df_io.index))
print(df.shape)

In [ ]:
for (columnName, columnData) in df.iteritems():
    print('Colunm Name : ', columnName )
    print('Colunm Name : ', df[columnName].dtypes )
    print('Number of unique values : ', df[columnName].nunique() ) 
    print('Column unique values : ', np.unique(df[columnName].astype(str)) )
    print('######')

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df)

In [ ]:
df.head(4)

In [ ]:
# have lost somhow pretty much of the data due to the sparcity and empty cols

In [ ]:
#lets add the time and date column
#CPUDT_MKPF 	CPUTM_MKPF
#df['CPUDT'] = df_re['CPUDT']

#as Timestamp
#df['TMSTMP'] = pd.to_datetime(df['CPUTM'].astype(str)+' '+df['CPUDT'].astype(str))
df['TMSTMP'] = pd.to_datetime(df['CPUDT'].astype(str) + df['CPUTM'].astype(str) , format='%Y-%m-%d%H:%M:%S')

In [ ]:
df.head(4)

In [ ]:
#sanity check
df.shape

In [ ]:
# delete plant specific prefixes

In [ ]:
df.drop(['BWKEY','BUKRS_x','BUKRS_y', 'WERKS'],axis=1, inplace=True)

In [ ]:
df.MATBF = df.MATBF.str.slice(3)
df.MATNR = df.MATNR.str.slice(3)

In [ ]:
df.head(4)

## data types correction

In [ ]:
df.describe(include='all').T

In [ ]:
df.sample(10)

In [ ]:
#2 Del
#ZEILE LINE_ID CPUTM_MKPF
df = df.drop(['BUZEI'], axis=1)

In [ ]:
cont_Cols = ['BELNR', 'EBELN', 'EBELP' ]

#to many we create this list later by computation
cat_Cols = [ ]

num_Cols = ['WRBTR', 'MENGE', 'BPMNG', 'LBKUM', 'SALK3', 'RMWWR' ]

dat_Cols = ['BLDAT', 'BUDAT', 'CPUTM', 'CPUDT', 'ZFBDT', 'WWERT', 'TMSTMP']



In [ ]:
list1 = df.columns.to_list()
print(list1)

#we already have these cols in num or cont or dat so lets make a list and remove them from all other than we have cat cols
unw = num_Cols + dat_Cols + cont_Cols

#cat cols are all cols - the cols we already defined as num / dat / cat
cat_Cols = [x for x in list1 if x not in unw]

In [ ]:
print(cat_Cols)

In [ ]:
#sanity check of cols
print(len(cont_Cols)+len(cat_Cols)+len(num_Cols)+len(dat_Cols))
print(df.shape[1]) 

In [ ]:
# Change dtype to Category for relevant features
df_cor = df.copy()
df_cor.head()

# we have to change NaN top 0 to avoid the infer dtype "object" for some cols
#df_cor['ZUONR'] = df_cor['ZUONR'].fillna(0)


# change Dtype to Cat for categorical columns
cats_cat = df_cor[cat_Cols]
        
for (columnName, columnData) in cats_cat.items():
    df_cor[columnName] = df_cor[columnName].astype(str)
    if isinstance(df_cor[columnName][0], str):
        s = pd.Series(df_cor[columnName], dtype="category")
        df_cor[columnName] = df_cor[columnName].astype('category')



In [ ]:
#check 1
df_cor.dtypes

In [ ]:
#check 3
df_cor.shape

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_cor)

In [ ]:
df_cor.to_feather('../Data/D3_RBKP-RSEG_raw.ftr')

# lets check the dataset properities / Correlation  / distribution

In [ ]:

# Compute the corr matrix and 
corr = df.corr(numeric_only=True).round(2)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(7, 7))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
df.sample(5)

In [ ]:
#manually delete the hihgly correlating features in both dataframes
# WRBTR PSWBT VBEL2 POSN2 ERFMG AUFPL SAKNR SAKNR LOKKT KIDNO
df_cor = df_cor.drop(['BPMNG'], axis=1)
df = df.drop(['BPMNG'], axis=1)

df_cor = df_cor.drop(['XBLNR_y'], axis=1)
df = df.drop(['XBLNR_y'], axis=1)


In [ ]:
df_cor.head()

In [ ]:
# Compute the corr matrix and 
corr = df.corr(numeric_only=True).round(2)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(10, 10))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
df_cor.to_feather('../Data/D3_RBKP-RSEG_cleaned.ftr')